In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


In [ ]:
import os
from langchain_community.document_loaders import WebBaseLoader

# 웹 요청을 위한 USER_AGENT 환경 변수 설정 (필요한 경우)
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"

# 환경 변수 확인
print(f"현재 설정된 USER_AGENT: {os.environ.get('USER_AGENT')}")

# 웹페이지 URL 지정  https://ko.wikipedia.org/wiki/축구_경기_규칙
url = 'https://ko.wikipedia.org/wiki/%EC%B6%95%EA%B5%AC_%EA%B2%BD%EA%B8%B0_%EA%B7%9C%EC%B9%99'

# WebBaseLoader 초기화 및 데이터 로드
loader = WebBaseLoader(url)
docs = loader.load()

# 로드된 문서 확인
print(type(docs), len(docs))
print(docs)
print(type(docs[0]))  # 
print(len(docs[0].page_content))
print(docs[0].page_content[5000:5500])
print(docs[0].metadata)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)

# 문서 분할
splits = text_splitter.split_documents(docs)

# 분할된 문서 확인
print(type(splits), len(splits))  # 총 몇 개의 청크가 생성되었는지 확인
print(type(splits[0]))
print(splits[0].page_content[:20])  # 첫 번째 청크의 일부 출력
# 열번째 청크의 내용 출력
print(splits[10].page_content[:20])
# 열번째 청크의 메타데이터 출력
print(splits[10].metadata)

In [ ]:
from langchain_community.vectorstores import FAISS  # 벡터 저장소 라이브러리
from langchain_openai import OpenAIEmbeddings # OpenAI의 임베딩(Embedding) 모델

# 1. FAISS 벡터 저장소 생성
# - documents: 텍스트 데이터를 벡터화 하여 저장할 문서 리스트
# - embedding: 문서를 벡터로 변환하는 OpenAI Embeddings 모델 사용
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 2. 유사 문서 검색 (Similarity Search)
# - "경기장 표시에 대해서 설명해주세요."라는 쿼리에 대해,
# - FAISS 벡터 저장소에서 가장 유사한 문서를 검색함.
docs = vectorstore.similarity_search("경기장 표시에 대해서 설명해주세요.")

# 3. 검색된 문서의 타입과 개수 출력
print(type(docs), len(docs))
# 4. 검색된 첫 번째 문서 내용 출력
print(docs[0].page_content)

In [6]:
from langchain_openai import ChatOpenAI  # OpenAI LLM(대화형 언어 모델)
from langchain_core.prompts import ChatPromptTemplate  # 프롬프트 템플릿
from langchain_core.runnables import RunnablePassthrough  # 입력을 그대로 전달하는 유틸리티
from langchain_core.output_parsers import StrOutputParser  # LLM 응답을 문자열로 변환하는 파서
from pprint import pprint

# 검색 개수 제한 설정
# - 벡터 저장소(vectorstore)에서 관련성이 높은 문서 최대 3개를 검색하도록 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})  

def format_docs(docs):
    summaries = [f"출처: {doc.metadata.get('source', '알 수 없음')}\n" + doc.page_content[:300] + "..." for doc in docs]
    return "\n\n".join(summaries)    

# 프롬프트 템플릿 설정
# - 모델이 주어진 `context`(검색된 문서)만을 참고하여 질문에 답변하도록 유도하는 프롬프트 템플릿
# - {context}: 검색된 문서 요약이 삽입될 자리
# - {question}: 사용자의 질문이 삽입될 자리
template = '''당신은 제공된 컨텍스트를 기반으로 질문에 답하는 AI 어시스턴트입니다. 
반드시 컨텍스트 내 정보를 활용하여 정확하고 신뢰할 수 있는 답변을 제공하세요.

[컨텍스트]
{context}

[질문]
{question}

[답변]
'''

# - 위에서 정의한 템플릿을 사용하여 LangChain의 프롬프트 객체 생성
prompt = ChatPromptTemplate.from_template(template)  

# LLM 모델 설정
# - OpenAI의 `gpt-3.5-turbo-0125` 모델 사용 (정확도를 높이기 위해 temperature=0 설정)
model = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

# RAG 체인 설정
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  
    # - `retriever`를 통해 검색된 문서를 `format_docs()` 함수로 가공하여 `context`로 전달
    # - `question`은 변형 없이 그대로 전달
    | prompt  # - 위에서 정의한 `ChatPromptTemplate`을 적용
    | model   # - OpenAI GPT-3.5 모델을 사용해 응답 생성
    | StrOutputParser()  # - 모델의 응답을 문자열로 변환
)

# 실행 (사용자 질문을 입력으로 받아 RAG 체인 실행)
# - "경기장 표시에 대해서 설명해주세요."라는 질문을 LLM에 전달하여 답변을 생성
response = rag_chain.invoke("경기장 표시에 대해서 설명해주세요.")  

# 최종 응답 출력
print(f" 모델 응답:\n")
pprint(response)

 모델 응답:

('경기장은 직사각형이어야 하며 위험하지 않아야 하는 연속적인 선으로 표시되어야 합니다. 경기장에는 규칙에 명시된 라인만 표시되어야 하며, '
 '인공 표면이 사용되는 경우 다른 색이 있고 축구 라인과 명확하게 구별되는 다른 라인이 허용됩니다. 두 개의 긴 경계선은 터치 라인이고, '
 '두 개의 짧은 라인은 골 라인입니다. 경기장은 두 개의 터치 라인의 중간 지점을 연결하는 중간 라인도 있어야 합니다.')


In [ ]:
# 개선된 RAG 파이프라인 - 축구 규칙 질의응답 시스템

import os
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

# 환경 설정
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"

print("=== 개선된 RAG 파이프라인 시작 ===\n")

# Step 1: 웹 데이터 로드
print(" Step 1: 웹페이지 데이터 로딩...")
url = 'https://ko.wikipedia.org/wiki/%EC%B6%95%EA%B5%AC_%EA%B2%BD%EA%B8%B0_%EA%B7%9C%EC%B9%99'
loader = WebBaseLoader(url)
docs = loader.load()
print(f" 로드된 문서 수: {len(docs)}")
print(f" 전체 텍스트 길이: {len(docs[0].page_content):,} 문자\n")

# Step 2: 개선된 문서 분할 (더 작은 청크, 더 많은 overlap)
print(" Step 2: 문서 분할 (개선된 설정)...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 기존 3000 → 1000으로 감소 (더 세밀한 검색)
    chunk_overlap=200,  # overlap 유지
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]  # 분할 우선순위 명시
)

splits = text_splitter.split_documents(docs)
print(f" 분할된 청크 수: {len(splits)} (기존 대비 증가)")
print(f" 첫 번째 청크 예시: {splits[0].page_content[:100]}...\n")

# Step 3: 개선된 벡터 저장소 생성 및 검색 설정
print(" Step 3: 벡터 저장소 생성 및 검색 설정...")
vectorstore = FAISS.from_documents(
    documents=splits, 
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")  # 최신 임베딩 모델 사용
)

# 개선된 검색 설정: 더 많은 문서 검색
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}  # 기존 3 → 6으로 증가
)
print(" 벡터 저장소 및 검색기 설정 완료\n")

# Step 4: 개선된 문서 포맷팅 함수
def format_docs(docs):
    """검색된 문서들을 LLM이 이해하기 쉬운 형태로 포맷팅"""
    formatted_docs = []
    for i, doc in enumerate(docs, 1):
        # 메타데이터에서 출처 정보 추출
        source = doc.metadata.get('source', '알 수 없음')
        
        # 문서 내용 정리 (불필요한 공백 제거)
        content = doc.page_content.strip()
        
        # 각 문서를 번호와 함께 명확히 구분
        formatted_doc = f"[문서 {i}]\n출처: {source}\n내용: {content}\n"
        formatted_docs.append(formatted_doc)
    
    return "\n" + "="*50 + "\n".join(formatted_docs) + "="*50 + "\n"

# Step 5: 개선된 프롬프트 템플릿
print(" Step 4: 개선된 프롬프트 설정...")

template = '''당신은 축구 규칙 전문가입니다. 아래 제공된 컨텍스트를 바탕으로 사용자의 질문에 정확하고 상세하게 답변해주세요.

**답변 지침:**
1. 제공된 컨텍스트의 정보만을 사용하여 답변하세요
2. 구체적인 수치, 규칙, 조건들을 포함하여 상세히 설명하세요
3. 관련된 여러 규칙이 있다면 체계적으로 정리해주세요
4. 컨텍스트에 없는 정보는 추측하지 마세요

**컨텍스트:**
{context}

**질문:** {question}

**답변:**
위 컨텍스트를 바탕으로 질문에 대해 상세히 답변드리겠습니다.

'''

prompt = ChatPromptTemplate.from_template(template)

# Step 6: 개선된 LLM 설정
print(" Step 5: LLM 모델 설정...")
model = ChatOpenAI(
    model='gpt-4o-mini',  # 더 강력한 모델 사용
    temperature=0.1,  # 약간의 창의성 허용하되 일관성 유지
    max_tokens=1500   # 더 길고 상세한 답변 허용
)

# Step 7: RAG 체인 구성
print(" Step 6: RAG 체인 구성...")
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(" RAG 파이프라인 설정 완료!\n")

# Step 8: 테스트 실행
def ask_question(question):
    """질문을 받아 RAG 시스템으로 답변 생성"""
    print(f" 질문: {question}")
    print("-" * 60)
    
    # 관련 문서 먼저 확인
    docs = retriever.invoke(question)
    print(f" 검색된 관련 문서 수: {len(docs)}")
    
    # RAG 체인 실행
    response = rag_chain.invoke(question)
    
    print(" 답변:")
    print(response)
    print("\n" + "="*80 + "\n")
    
    return response

# 테스트 질문들
if __name__ == "__main__":
    test_questions = [
        "경기장 표시에 대해서 설명해주세요.",
        "페널티 에어리어의 크기와 규격은 어떻게 되나요?",
        "오프사이드 규칙에 대해 자세히 설명해주세요.",
        "축구공의 규격과 요구사항은 무엇인가요?",
        "골키퍼가 할 수 없는 행동들은 무엇인가요?"
    ]
    
    for question in test_questions:
        ask_question(question)